In [1]:
!pip install dataset
!pip install trl
!pip install peft
!pip install transformers==4.40.0
!pip install accelerate==0.29.3
!pip install accelerated-scan==0.1.2
!pip install flash-attn==2.5.7
!pip install mamba-ssm==1.2.0.post1
!pip install causal-conv1d==1.2.0.post1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 52.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━

In [11]:
!pip install transformers==4.40.2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.40.2-py3-none-any.whl.metadata (137 kB)
Using cached transformers-4.40.2-py3-none-any.whl (9.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [1]:
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
import pickle
from datasets import Dataset
from transformers import default_data_collator
from tqdm import tqdm
from transformers import BitsAndBytesConfig
import torch
from huggingface_hub import login

In [2]:
login(token = "#")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


#Must use high RAM. T4 is enough.

In [3]:
'''
#Mamba 2.8B
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-2.8b-hf")
model = AutoModelForCausalLM.from_pretrained("state-spaces/mamba-2.8b-hf")
del model
del tokenizer
'''
#Mamba 7B
tokenizer = AutoTokenizer.from_pretrained("tri-ml/mamba-7b-rw",torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("tri-ml/mamba-7b-rw",torch_dtype=torch.float16)
#del model
#del tokenizer
'''
#Mamba 1.4B
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-1.4b-hf",torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("state-spaces/mamba-1.4b-hf",torch_dtype=torch.float16)
del model
del tokenizer

#LLAMA2-7B
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf",torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf",torch_dtype=torch.float16)

del model
del tokenizer
'''
#Mistral-7B
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1",torch_dtype=torch.float16)
#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",torch_dtype=torch.float16)

#del model
#del tokenizer
'''
#LLAMA3 8B Does not run on T4
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B",torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B",torch_dtype=torch.float16)

del model
del tokenizer

#stabilityai/
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-zephyr-3b",torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-zephyr-3b",torch_dtype=torch.float16)

del model
del tokenizer

#Recurrent Gemma
tokenizer = AutoTokenizer.from_pretrained("google/recurrentgemma-2b",torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("google/recurrentgemma-2b",torch_dtype=torch.float16)
print('here')
del model
del tokenizer

#Gemma 2B
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b",torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b",torch_dtype=torch.float16)

del model
del tokenizer
'''
#phi 1.5B 

#tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5",torch_dtype=torch.float16)
#model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5",torch_dtype=torch.float16)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at tri-ml/mamba-7b-rw were not used when initializing MambaForCausalLM: ['model.lm_head.weight']
- This IS expected if you are initializing MambaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MambaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model

'\n#LLAMA3 8B Does not run on T4\ntokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B",torch_dtype=torch.float16)\nmodel = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B",torch_dtype=torch.float16)\n\ndel model\ndel tokenizer\n\n#stabilityai/\ntokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-zephyr-3b",torch_dtype=torch.float16)\nmodel = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-zephyr-3b",torch_dtype=torch.float16)\n\ndel model\ndel tokenizer\n\n#Recurrent Gemma\ntokenizer = AutoTokenizer.from_pretrained("google/recurrentgemma-2b",torch_dtype=torch.float16)\nmodel = AutoModelForCausalLM.from_pretrained("google/recurrentgemma-2b",torch_dtype=torch.float16)\nprint(\'here\')\ndel model\ndel tokenizer\n\n#Gemma 2B\ntokenizer = AutoTokenizer.from_pretrained("google/gemma-2b",torch_dtype=torch.float16)\nmodel = AutoModelForCausalLM.from_pretrained("google/gemma-2b",torch_dtype=torch.float16)\n\ndel model\ndel tokenizer\n'

#Inference

In [4]:
model = model.to('cuda:0')

In [5]:
model.generation_config.pad_token_id = model.generation_config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'right'
model = model.eval()

In [12]:
inputs = tokenizer("You need to write a story about a beautiful girl, Make the story as long as possible. Start Story: ", return_tensors="pt").to(model.device)


In [13]:
from typing import Any, Dict, Optional, Tuple, Union
from transformers import MambaConfig
class MambaCache:
    """
    Arguments:
        config: MambaConfig
        batch_size: int
        dtype: torch.dtype
        device: torch.device

    Attributes:
        seqlen_offset: int
        dtype: torch.dtype
        conv_states: Dict[int, torch.Tensor] # layer_idx -> [batch_size, intermediate_size, conv_kernel_size]
        ssm_states: Dict[int, torch.Tensor] # layer_idx -> [batch_size, intermediate_size, ssm_state_size]
    """

    def __init__(
        self, config: MambaConfig, batch_size: int, dtype: torch.dtype = torch.float16, device: Optional[str] = None
    ):
        self.seqlen_offset = 0
        self.dtype = dtype
        intermediate_size = config.intermediate_size
        ssm_state_size = config.state_size
        conv_kernel_size = config.conv_kernel

        self.conv_states = {
            i: torch.zeros(batch_size, intermediate_size, conv_kernel_size, device=device, dtype=dtype)
            for i in range(config.num_hidden_layers)
        }
        self.ssm_states = {
            i: torch.zeros(batch_size, intermediate_size, ssm_state_size, device=device, dtype=dtype)
            for i in range(config.num_hidden_layers)
        }


In [16]:
model.generate(inputs['input_ids'], max_length=128, cache_params =MambaCache(model.config, 1, device = model.device))

tensor([[ 1394,   878,   281,  3630,   247,  2926,   670,   247,  5389,  3226,
            13, 10338,   253,  2926,   347,  1048,   347,  1896,    15, 11075,
         16061,    27,   209,   187,    14,   370,    18,    13,    25,  2222,
          8979,    67,   412,    15,   681, 33397,  5382,    34,    15, 26006,
          9552,    27,  2596, 19662,   187,    14,   370,    18,    13,    25,
          2222,  8979,    67,   412,    15,   681, 33397,  5382,    34,    15,
         26006,  9552,    27,  2596, 19662,   187,    14,   370,    18,    13,
            25,  2222,  8979,    67,   412,    15,   681, 33397,  5382,    34,
            15, 26006,  9552,    27,  2596, 19662,   187,    14,   370,    18,
            13,    25,  2222,  8979,    67,   412,    15,   681, 33397,  5382,
            34,    15, 26006,  9552,    27,  2596, 19662,   187,    14,   370,
            18,    13,    25,  2222,  8979,    67,   412,    15,   681, 33397,
          5382,    34,    15, 26006,  9552,    27,  

In [8]:
print(len(outputs[0].tolist()))

512


In [9]:
print(tokenizer.decode(outputs[0]))

You need to write a story about a beautiful girl, Make the story as long as possible. Start Story: *************** Once upon a time there was a beautiful girl. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. She was very beautiful. 

#Extracting Embading

In [5]:
from transformers import AutoTokenizer, AutoModel

In [56]:
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-zephyr-3b",torch_dtype=torch.float16)
model = AutoModel.from_pretrained("stabilityai/stablelm-zephyr-3b",torch_dtype=torch.float16)
#tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-1.4b-hf",torch_dtype=torch.float16)
#model = AutoModelForCausalLM.from_pretrained("state-spaces/mamba-1.4b-hf",torch_dtype=torch.float16)
#tokenizer = AutoTokenizer.from_pretrained("google/recurrentgemma-2b",torch_dtype=torch.float16)
#model = AutoModelForCausalLM.from_pretrained("google/recurrentgemma-2b",torch_dtype=torch.float16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [57]:
model = model.to('cuda:0')

In [58]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt").to(model.device)
model = model.eval()
outputs = model(inputs['input_ids'],output_hidden_states =True)

#Final Layer Embeding

In [59]:
#outputs.last_hidden_state #for transformer
outputs["hidden_states"][-1][0,-1,:].shape #for mambas

torch.Size([2560])

#Word Embeding

In [35]:
model.get_input_embeddings()(inputs['input_ids'])

tensor([[[ 0.0549, -0.2002, -0.0786,  ..., -0.0874,  0.0771,  0.0222],
         [-0.0449, -0.0140,  0.0070,  ...,  0.1094,  0.1152, -0.0522],
         [ 0.0435,  0.0062,  0.2637,  ...,  0.0903,  0.0664, -0.0430],
         ...,
         [-0.0693,  0.0723,  0.0747,  ...,  0.1187, -0.0236, -0.0938],
         [-0.1455,  0.0796,  0.0015,  ...,  0.1367,  0.0603, -0.0045],
         [ 0.0547,  0.0025, -0.0640,  ...,  0.1270, -0.0320,  0.1436]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<EmbeddingBackward0>)